In [ ]:
import csv
import cv2
import numpy as np

def read_paths(paths):
    imgs = []
    for path in paths:
        with open(path) as csvf:
            readfile=csv.reader(csvf)
            for line in readfile:
                imgs.append(line)
    return imgs
data=['data/driving_log.csv', 'data2/driving_log.csv',                           
      'data_offroad_correct/driving_log.csv', 'data_shadow_track2/driving_log.csv',
      'data_track2/driving_log.csv', 'data_track2_2/driving_log.csv', 'data_curve/driving_log.csv']                                                                               
data = read_paths(data)




In [ ]:
import sklearn
from random import shuffle
def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                #flip half the images
                #if i%2:
                center_image = cv2.imread(batch_sample[0])
                center_angle = float(batch_sample[3])
                #else:
                center_image = np.fliplr(cv2.imread(batch_sample[0]))
                center_angle = -float(batch_sample[3])
                images.append(center_image)
                angles.append(center_angle)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)


In [ ]:
from keras.models import Sequential                                              
from keras.layers import Flatten, Dense, Conv2D, Lambda, Cropping2D              
from keras.layers.core import Dropout                                            
from keras.layers.advanced_activations import PReLU                              
                                                                            
from sklearn.model_selection import train_test_split                             
train_samples, valid_samples = train_test_split(data, test_size=0.2)             
rate = 0.25                                                                      
rateconv = 0.25                                                                  
batch_size = 16                                                                  
                                                                                 
train_generator=generator(train_samples, batch_size=batch_size)                  
valid_generator=generator(valid_samples, batch_size=batch_size)                  

nn = Sequential()                                                                
nn.add(Cropping2D(cropping=((50,0), (0,0)), input_shape=(160,320,3)))            
nn.add(Lambda(lambda x : x/126-1))                                               
nn.add(Conv2D(24, (15, 15), strides=(3,6), activation = 'relu'))                 
nn.add(Dropout(rateconv))                                                        
nn.add(Conv2D(48, (7,7), strides=(3,3), activation='relu'))                      
nn.add(Dropout(rateconv))                                                        
nn.add(Conv2D(96, (5,5), activation='relu'))                                     
nn.add(Dropout(rateconv))                                                        
nn.add(Conv2D(192, (3,3), activation='relu'))                                    
nn.add(Dropout(rateconv))                                                        
nn.add(Flatten())                                                                
nn.add(Dense(1))                                                                 
nn.compile(loss='mse', optimizer='adam')                                         
nn.fit_generator(train_generator, steps_per_epoch=                               
             len(train_samples)/batch_size+ len(train_samples)%batch_size, validation_data=valid_generator,
             validation_steps=len(valid_samples)/batch_size + len(valid_samples)%batch_size, epochs=5)

nn.save('dr8_5.h5') 


In [ ]:
#nn.fit(images, angles, validation_split=0.2, shuffle=True, epochs=15)
nn.fit_generator(train_generator, steps_per_epoch=
            len(train_samples)/batch_size+ len(train_samples)%batch_size, validation_data=valid_generator,
            validation_steps=len(valid_samples)/batch_size + len(valid_samples)%batch_size, epochs=15)
nn.save('dr8_20.h5')

In [ ]:
#nn.fit(images, angles, validation_split=0.2, shuffle=True, epochs=20)
nn.fit_generator(train_generator, steps_per_epoch=
            len(train_samples)/batch_size+ len(train_samples)%batch_size, validation_data=valid_generator,
            validation_steps=len(valid_samples)/batch_size + len(valid_samples)%batch_size, epochs=20)
nn.save('dr8_40.h5')

In [ ]:
#nn.fit(images, angles, validation_split=0.2, shuffle=True, epochs=20)
nn.fit_generator(train_generator, steps_per_epoch=
            len(train_samples)/batch_size+ len(train_samples)%batch_size, validation_data=valid_generator,
            validation_steps=len(valid_samples)/batch_size + len(valid_samples)%batch_size, epochs=20)
nn.save('dr8_60.h5')

In [ ]:
#nn.fit(images, angles, validation_split=0.2, shuffle=True, epochs=20)
nn.fit_generator(train_generator, steps_per_epoch=
            len(train_samples)/batch_size+ len(train_samples)%batch_size, validation_data=valid_generator,
            validation_steps=len(valid_samples)/batch_size + len(valid_samples)%batch_size, epochs=20)
nn.save('dr8_80.h5')

In [ ]:
#nn.fit(images, angles, validation_split=0.2, shuffle=True, epochs=20)
nn.fit_generator(train_generator, steps_per_epoch=
            len(train_samples)/batch_size+ len(train_samples)%batch_size, validation_data=valid_generator,
            validation_steps=len(valid_samples)/batch_size + len(valid_samples)%batch_size, epochs=60)
nn.save('dr8_100.h5')